## Data Import

In [1]:
import pandas as pd


In [8]:
heartFailure=pd.read_csv("heart_failure_clinical_records_dataset.csv")
heartFailure.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [9]:
heartFailure.shape

(299, 13)

## Data Description:

Ejection_fraction = Ejection fraction (EF) is a measurement, expressed as a percentage,
                     of how much blood the left ventricle pumps out with each contraction. 

anaemia: decrease of red blood cells or hemoglobin (boolean)
high blood pressure: if the patient has hypertension (boolean)
 
creatinine_phosphokinase =  an enzyme in the body. It is found mainly in the heart, brain, and skeletal muscle.
                            The CPK normal range for a male is between 39 – 308 U/L, 
                            while in females the CPK normal range is between 26 – 192 U/L.
 
Platelets = The cells that circulate within our blood and bind together when they recognize damaged blood vessels.
             A normal platelet count ranges from 150,000 to 450,000 platelets per microliter of blood.

serum_creatinine = An increased level of creatinine may be a sign of poor kidney function.
                    The typical range for serum creatinine is: For adult men, 0.74 to 1.35 mg/dL 
                    For adult women, 0.59 to 1.04 mg/dL (52.2 to 91.9 micromoles/L)

serum_sodium = A measurement in assessing electrolyte, acid-base, and water balance, as well as renal function.
               The reference range for serum sodium is 135-147 mmol/L.
sex: woman[0] or man[1] (binary)
smoking: if the patient smokes or not (boolean)
time: follow-up period (days)
[target] death event: if the patient deceased during the follow-up period (boolean)